## Generalized Indices ground truth for SSZ-QL GI calculation
The target of this Notebook is to support the development of Generalized Indices calculation in the scope of Generalized Merkle Multiproofs

In [1]:
from eth2spec.electra.mainnet import get_generalized_index, BeaconState
from eth2spec.utils.ssz.ssz_typing import Container, List, Vector, Bytes32, uint64, byte, Bitlist, Bitvector, uint32, boolean
from typing import Union as PyUnion
from remerkleable.core import View

In [2]:
from remerkleable.core import View, ViewMeta
from remerkleable.complex import Container as RemerkleableContainer

def is_container(obj):
    """Check if an object is a Container class."""
    try:
        if isinstance(obj, type) and isinstance(obj, ViewMeta):
            # Check if it's a Container by looking at its methods
            return hasattr(obj, 'fields') and callable(obj.fields)
        return False
    except:
        return False

def get_container_fields(container_class):
    """Get the fields of a Container class as a dict."""
    try:
        if hasattr(container_class, 'fields') and callable(container_class.fields):
            return container_class.fields()
        return {}
    except:
        return {}

def is_list_type(field_type):
    """Check if a field type is a List."""
    try:
        type_name = getattr(field_type, '__name__', '')
        type_str = str(field_type)
        return 'List' in type_str or (hasattr(field_type, '__origin__') and hasattr(field_type.__origin__, '__name__') and 'List' in field_type.__origin__.__name__)
    except:
        return False

def is_vector_type(field_type):
    """Check if a field type is a Vector."""
    try:
        type_str = str(field_type)
        return 'Vector' in type_str
    except:
        return False

def traverse_container_fields(container_class, path=None, depth=0, root_container=None):
    """
    Recursively traverse all fields and nested fields in a Container.
    Print field names and their generalized indices with tab indentation.
    """
    if path is None:
        path = []
    if root_container is None:
        root_container = container_class
    
    fields = get_container_fields(container_class)
    
    for field_name, field_type in fields.items():
        current_path = path + [field_name]
        indent = "\t" * depth
        
        try:
            # Get GI for this field from the root container
            gi = get_generalized_index(root_container, *current_path)
            print(f"{indent}{field_name}: {gi}")
            
            # If it's a Container, recurse into it
            if is_container(field_type):
                traverse_container_fields(field_type, current_path, depth + 1, root_container)
            # If it's a List of Containers, show list element type info
            elif is_list_type(field_type):
                # Try to get element type and recurse if it's a container
                elem_type = None
                try:
                    # For remerkleable lists, try to access element_cls
                    if hasattr(field_type, 'element_cls'):
                        elem_type = field_type.element_cls()
                    elif hasattr(field_type, '__args__'):
                        elem_type = field_type.__args__[0]
                except:
                    pass
                
                if elem_type and is_container(elem_type):
                    indent_nested = "\t" * (depth + 1)
                    print(f"{indent_nested}[element]:")
                    traverse_container_fields(elem_type, current_path + [0], depth + 2, root_container)
                
                # Show length GI
                try:
                    len_gi = get_generalized_index(root_container, *(current_path + ["__len__"]))
                    indent_len = "\t" * (depth + 1)
                    print(f"{indent_len}__len__: {len_gi}")
                except:
                    pass
        except Exception as e:
            indent = "\t" * depth
            print(f"{indent}{field_name}: [ERROR] {str(e)}")

def traverse_beacon_state(container_class):
    """Entry point for traversing BeaconState."""
    traverse_container_fields(container_class)

In [3]:
print("=" * 80)
print("BeaconState Field Hierarchy with Generalized Indices")
print("=" * 80)
print()
traverse_beacon_state(BeaconState)

BeaconState Field Hierarchy with Generalized Indices

genesis_time: 64
genesis_validators_root: 65
slot: 66
fork: 67
	previous_version: 268
	current_version: 269
	epoch: 270
latest_block_header: 68
	slot: 544
	proposer_index: 545
	parent_root: 546
	state_root: 547
	body_root: 548
block_roots: 69
state_roots: 70
historical_roots: 71
	__len__: 143
eth1_data: 72
	deposit_root: 288
	deposit_count: 289
	block_hash: 290
eth1_data_votes: 73
	[element]:
		deposit_root: 1196032
		deposit_count: 1196033
		block_hash: 1196034
	__len__: 147
eth1_deposit_index: 74
validators: 75
	[element]:
		pubkey: 1319413953331200
		withdrawal_credentials: 1319413953331201
		effective_balance: 1319413953331202
		slashed: 1319413953331203
		activation_eligibility_epoch: 1319413953331204
		activation_epoch: 1319413953331205
		exit_epoch: 1319413953331206
		withdrawable_epoch: 1319413953331207
	__len__: 151
balances: 76
	__len__: 153
randao_mixes: 77
slashings: 78
previous_epoch_participation: 79
	__len__: 159
curr